# Bitcoin Trading Strategy with Nadaraya-Watson Envelopes and EMA

## 1. Import required libraries

In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import requests
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.nonparametric.kernel_regression import KernelReg
from backtesting import Strategy, Backtest
import websocket
import json
import threading

## 2. Data Fetching Functions

In [2]:
def fetch_historical_data(symbol='BTCUSDT', interval='15m', limit=10000):
    """
    Fetch historical data from Binance API.

    Args:
    symbol (str): Trading pair symbol (default: 'BTCUSDT')
    interval (str): Candlestick interval (default: '15m')
    limit (int): Number of candlesticks to fetch (default: 10000)

    Returns:
    pd.DataFrame: Historical price data
    """
    base_url = 'https://api.binance.com'
    endpoint = f'/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }

    url = f'{base_url}{endpoint}'
    response = requests.get(url, params=params)
    data = response.json()

    df = pd.DataFrame(data, columns=[
        'timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'close_time',
        'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])

    df['Gmt time'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('Gmt time', inplace=True)

    df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

# %%
# WebSocket connection to Binance for live data
def on_message(ws, message):
    global df_live
    data = json.loads(message)
    candle = data['k']
    is_candle_closed = candle['x']

    if is_candle_closed:
        timestamp = pd.to_datetime(candle['t'], unit='ms')
        open_price = float(candle['o'])
        high_price = float(candle['h'])
        low_price = float(candle['l'])
        close_price = float(candle['c'])
        volume = float(candle['v'])

        new_row = pd.DataFrame({
            'Open': [open_price],
            'High': [high_price],
            'Low': [low_price],
            'Close': [close_price],
            'Volume': [volume]
        }, index=[timestamp])

        df_live = pd.concat([df_live, new_row])
        print(df_live.tail(1))  # Print the last row for debugging

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    print("### connection opened ###")

def run_live_stream():
    symbol = 'btcusdt'
    socket = f"wss://stream.binance.com:9443/ws/{symbol}@kline_15m"
    ws = websocket.WebSocketApp(socket,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.on_open = on_open
    ws.run_forever()

## 3. Technical Indicators and Nadaraya-Watson Calculations

In [3]:
def calculate_indicators(df):
    """
    Calculate technical indicators: EMA and ATR.

    Args:
    df (pd.DataFrame): Price data

    Returns:
    pd.DataFrame: DataFrame with added indicator columns
    """
    df["EMA_slow"] = ta.ema(df.Close, length=50)
    df["EMA_fast"] = ta.ema(df.Close, length=40)
    df['ATR'] = ta.atr(df.High, df.Low, df.Close, length=7)
    return df

# %%
def calculate_nadaraya_watson(df, bandwidth=0.1):
    """
    Calculate Nadaraya-Watson envelopes.

    Args:
    df (pd.DataFrame): Price data with indicators
    bandwidth (float): Bandwidth for the kernel, controls smoothness

    Returns:
    pd.DataFrame: DataFrame with added Nadaraya-Watson columns
    """
    # Ensure we're working with a copy to avoid overwriting the original data
    df = df.copy()

    # Convert datetime index to numerical values
    X = np.arange(len(df)).reshape(-1, 1)
    y = df['Close'].values

    # Perform Nadaraya-Watson kernel regression
    model = KernelReg(endog=y, exog=X, var_type='c', bw=[bandwidth])
    fitted_values, _ = model.fit(X)

    # Store the fitted values
    df['NW_Fitted'] = fitted_values

    # Calculate the residuals
    residuals = df['Close'] - fitted_values

    # Calculate the standard deviation of the residuals
    std_dev = np.std(residuals)

    # Create the envelopes
    df['Upper_Envelope'] = df['NW_Fitted'] + 2 * std_dev
    df['Lower_Envelope'] = df['NW_Fitted'] - 2 * std_dev

    # Print some debug information
    print("NW_Fitted range:", df['NW_Fitted'].min(), "-", df['NW_Fitted'].max())
    print("Close range:", df['Close'].min(), "-", df['Close'].max())
    print("Std Dev:", std_dev)
    print("Upper_Envelope range:", df['Upper_Envelope'].min(), "-", df['Upper_Envelope'].max())
    print("Lower_Envelope range:", df['Lower_Envelope'].min(), "-", df['Lower_Envelope'].max())

    return df

## 4. Signal Generation

In [4]:
def ema_signal(df, backcandles):
    """
    Generate EMA crossover signals.

    Args:
    df (pd.DataFrame): Price data with indicators
    backcandles (int): Number of candles to look back for confirming the signal

    Returns:
    pd.DataFrame: DataFrame with added EMA signal column
    """
    above = df['EMA_fast'] > df['EMA_slow']
    below = df['EMA_fast'] < df['EMA_slow']

    above_all = above.rolling(window=backcandles).apply(lambda x: x.all(), raw=True).fillna(0).astype(bool)
    below_all = below.rolling(window=backcandles).apply(lambda x: x.all(), raw=True).fillna(0).astype(bool)

    df['EMASignal'] = 0
    df.loc[above_all, 'EMASignal'] = 2
    df.loc[below_all, 'EMASignal'] = 1

    return df

# %%
def total_signal(df):
    """
    Generate total trading signals based on EMA and Nadaraya-Watson envelopes.

    Args:
    df (pd.DataFrame): Price data with indicators and EMA signals

    Returns:
    pd.DataFrame: DataFrame with added total signal column
    """
    condition_buy = (df['EMASignal'] == 2) & (df['Close'] <= df['Lower_Envelope'])
    condition_sell = (df['EMASignal'] == 1) & (df['Close'] >= df['Upper_Envelope'])

    df['Total_Signal'] = 0
    df.loc[condition_buy, 'Total_Signal'] = 2
    df.loc[condition_sell, 'Total_Signal'] = 1

    return df

## 5. Visualization

In [5]:
def plot_chart(df):
    """
    Create an interactive chart with candlesticks, EMAs, and Nadaraya-Watson envelopes.

    Args:
    df (pd.DataFrame): Price data with all indicators and signals
    """
    fig = make_subplots(rows=1, cols=1)

    # Candlestick chart
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'],
                                 name='BTC/USDT'))

    # EMAs
    fig.add_trace(go.Scatter(x=df.index, y=df['EMA_fast'], name='EMA Fast', line=dict(color='blue', width=1)))
    fig.add_trace(go.Scatter(x=df.index, y=df['EMA_slow'], name='EMA Slow', line=dict(color='orange', width=1)))

    # Nadaraya-Watson envelopes
    fig.add_trace(go.Scatter(x=df.index, y=df['Upper_Envelope'], name='Upper Envelope', line=dict(color='red', width=1)))
    fig.add_trace(go.Scatter(x=df.index, y=df['Lower_Envelope'], name='Lower Envelope', line=dict(color='green', width=1)))

    # Buy and Sell signals
    buy_signals = df[df['Total_Signal'] == 2]
    sell_signals = df[df['Total_Signal'] == 1]

    fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Low'], mode='markers',
                             marker=dict(symbol='triangle-up', size=10, color='green'), name='Buy Signal'))
    fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['High'], mode='markers',
                             marker=dict(symbol='triangle-down', size=10, color='red'), name='Sell Signal'))

    fig.update_layout(title='BTC/USDT Chart with Signals',
                      xaxis_title='Date',
                      yaxis_title='Price (USDT)',
                      xaxis_rangeslider_visible=False)

    fig.show()


## 6. Trading Strategy and Backtesting

In [19]:
class NWStrategy(Strategy):
    mysize = 1.0
    slcoef = 1.2
    TPSLRatio = 1.5

    def init(self):
        super().init()
        self.signal = self.I(lambda: self.data.Total_Signal)

    def next(self):
        super().next()
        slatr = self.slcoef * self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        if self.signal == 2 and not self.position:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr * TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal == 1 and not self.position:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr * TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

## 7. Main Execution

In [20]:
# Fetch historical data
df = fetch_historical_data(symbol='BTCUSDT', interval='15m')

# Calculate indicators and signals
df = calculate_indicators(df)
df = calculate_nadaraya_watson(df, 7)
df = ema_signal(df, backcandles=7)
df = total_signal(df)

# Plot the chart
plot_chart(df)

# Perform backtesting
bt = Backtest(df, NWStrategy, cash=200000, commission=.002, margin=1/75)
stats = bt.run()
print(stats)

# # Initialize live data streaming
# df_live = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
# ws_thread = threading.Thread(target=run_live_stream)
# ws_thread.start()

print("Live data streaming has started. Press Ctrl+C to stop.")

NW_Fitted range: 53599.2038653653 - 59269.911022507746
Close range: 52887.99 - 59764.0
Std Dev: 190.5667226844045
Upper_Envelope range: 53980.33731073411 - 59651.044467876556
Lower_Envelope range: 53218.07041999649 - 58888.777577138935


Start                     2024-08-31 09:15:00
End                       2024-09-10 19:00:00
Duration                     10 days 09:45:00
Exposure Time [%]                         8.2
Equity Final [$]                202866.581287
Equity Peak [$]                 202866.581287
Return [%]                           1.433291
Buy & Hold Return [%]               -2.453675
Return (Ann.) [%]                   60.354341
Volatility (Ann.) [%]                4.510524
Sharpe Ratio                        13.380784
Sortino Ratio                             inf
Calmar Ratio                       170.805694
Max. Drawdown [%]                   -0.353351
Avg. Drawdown [%]                   -0.119964
Max. Drawdown Duration        1 days 09:45:00
Avg. Drawdown Duration        0 days 05:17:00
# Trades                                   11
Win Rate [%]                        81.818182
Best Trade [%]                       1.182679
Worst Trade [%]                     -0.921205
Avg. Trade [%]                    

/usr/local/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning:

divide by zero encountered in scalar divide

